In [54]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from time import sleep
from tqdm import tqdm

In [55]:
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [56]:
# 提取Link
def extract_paper_links(url):
    paper_links = []
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    # 找到所有 <dt> 标签
    dt_tags = soup.find_all('dt')
    # 提取链接
    for dt_tag in dt_tags:
        a_tag = dt_tag.find('a', title='Abstract')
        if a_tag and 'href' in a_tag.attrs:
            link = a_tag.get('href')
            paper_links.append(link)            
    return paper_links

urls = []
for skip in range(0, 76, 25):
    url = f'https://arxiv.org/list/cs/pastweek?skip={skip}&show=25'
    urls.append(url)

all_paper_links = []
for url in tqdm(urls, desc="Processing pages", unit="page"):
    paper_links = extract_paper_links(url)
    all_paper_links.extend(paper_links)
    sleep(1)

# 打印所有链接
print(all_paper_links)

Processing pages: 100%|██████████| 4/4 [00:10<00:00,  2.74s/page]

['/abs/2403.00762', '/abs/2403.00758', '/abs/2403.00752', '/abs/2403.00745', '/abs/2403.00743', '/abs/2403.00742', '/abs/2403.00737', '/abs/2403.00729', '/abs/2403.00725', '/abs/2403.00724', '/abs/2403.00720', '/abs/2403.00717', '/abs/2403.00715', '/abs/2403.00712', '/abs/2403.00704', '/abs/2403.00696', '/abs/2403.00691', '/abs/2403.00690', '/abs/2403.00689', '/abs/2403.00686', '/abs/2403.00685', '/abs/2403.00682', '/abs/2403.00680', '/abs/2403.00675', '/abs/2403.00674', '/abs/2403.00673', '/abs/2403.00669', '/abs/2403.00668', '/abs/2403.00665', '/abs/2403.00663', '/abs/2403.00662', '/abs/2403.00646', '/abs/2403.00645', '/abs/2403.00644', '/abs/2403.00643', '/abs/2403.00642', '/abs/2403.00641', '/abs/2403.00633', '/abs/2403.00632', '/abs/2403.00631', '/abs/2403.00628', '/abs/2403.00625', '/abs/2403.00623', '/abs/2403.00622', '/abs/2403.00621', '/abs/2403.00613', '/abs/2403.00611', '/abs/2403.00607', '/abs/2403.00606', '/abs/2403.00598', '/abs/2403.00594', '/abs/2403.00592', '/abs/2403.

In [57]:
# 打开新的link
def extract_content(url):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    abs_div = soup.find('div', id='abs')
    
    # 提取标题
    title = ""
    title_tag = abs_div.find('h1', class_='title mathjax')
    if title_tag:
        title = title_tag.text.strip().replace("Title:", "").strip()
    
    # 提取作者
    authors_list = []
    authors_div = abs_div.find('div', class_='authors')
    if authors_div:
        for author_tag in authors_div.find_all('a'):
            authors_list.append(author_tag.text.strip())
    
    # 提取摘要
    abstract = ""
    abstract_blockquote = abs_div.find('blockquote', class_='abstract mathjax')
    if abstract_blockquote:
        abstract = abstract_blockquote.text.strip().replace("Abstract:", "").strip()
    
    # 提取领域    
    subject = ""
    subjects_td = abs_div.find('td', class_='tablecell subjects')
    if subjects_td:
        subject = subjects_td.find('span', class_='primary-subject').text.strip()

    return title, subject, ", ".join(authors_list), abstract

In [58]:
df = pd.DataFrame(columns=["Title", "Subjects", "Authors", "Abstract"])
for link in tqdm(all_paper_links, desc="Processing papers", unit="paper"):
    # 构造完整的URL
    full_url = 'https://arxiv.org/' + link
    df.loc[len(df)] = extract_content(full_url)
    sleep(1)
df

Processing papers: 100%|██████████| 100/100 [02:35<00:00,  1.56s/paper]


,Title,Subjects,Authors,Abstract
0,Point Could Mamba: Point Cloud Learning via St...,Computer Vision and Pattern Recognition (cs.CV),"Tao Zhang, Xiangtai Li, Haobo Yuan, Shunping J...","In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware P...,Computation and Language (cs.CL),"Qingyan Guo, Rui Wang, Junliang Guo, Xu Tan, J...",While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video Str...,Multimedia (cs.MM),"Imran Khan, Tuyen X. Tran, Matti Hiltunen, The...",Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for loc...,Machine Learning (cs.LG),"János Kramár, Tom Lieberum, Rohin Shah, Neel N...",Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pre...,"Distributed, Parallel, and Cluster Computing (...","Joshua Dennis Booth, Hongyang Sun, Trevor Garnett",The solution of a sparse system of linear equa...
...,...,...,...,...
95,Do Zombies Understand? A Choose-Your-Own-Adven...,Computation and Language (cs.CL),"Ariel Goldstein, Gabriel Stanovsky",Recent advances in LLMs have sparked a debate ...
96,"Graph Homomorphism, Monotone Classes and Bound...",Computational Complexity (cs.CC),"Tala Eagling-Vose, Barnaby Martin, Daniel Paul...",A recent paper describes a framework for study...
97,Analyzing Divergence for Nondeterministic Prob...,Logic in Computer Science (cs.LO),"Hao Wu, Yuxi Fu, Huan Long, Xian Xu, Wenbo Zhang",Branching and weak probabilistic bisimilaritie...
98,Multiple Ways of Working with Users to Develop...,Human-Computer Interaction (cs.HC),"Amal Nanavati, Max Pascher, Vinitha Ranganeni,...",Despite the growth of physically assistive rob...


In [60]:
df.to_csv('arxiv_papers.csv', index=False)